In [1]:
import warnings
warnings.simplefilter(action= 'ignore', category=FutureWarning)

import boto3
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn import tree


##defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'rachaeld-data445'
bucket = s3.Bucket(bucket_name)

#defining the csv file
file_key = 'train.csv'
file_key2 = 'test.csv'

##train
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##test
bucket_object2 = bucket.Object(file_key2)
file_object2 = bucket_object2.get()
file_content_stream2 = file_object2.get('Body')

train = pd.read_csv(file_content_stream)
test = pd.read_csv(file_content_stream2)

In [2]:
train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemsVoids,scansWithoutRegistration,quanitityModification,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


In [3]:
test.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
0,4.0,467.0,88.48,4,8,4,0.014989,0.189465,0.571429
1,3.0,1004.0,58.99,7,6,1,0.026892,0.058755,0.259259
2,1.0,162.0,14.00,4,5,4,0.006173,0.086420,4.000000
3,5.0,532.0,84.79,9,3,4,0.026316,0.159380,0.642857
4,5.0,890.0,42.16,4,0,0,0.021348,0.047371,0.210526


In [4]:
## engineering variables from homework 4
##boxcox
transformed_trustLevel= boxcox(train['trustLevel'])
train['newtrustLevel'] = transformed_trustLevel[0]
transformed_trustLevel= boxcox(test['trustLevel'])
test['newtrustLevel'] = transformed_trustLevel[0]

##1/x
train['1/scannedLineItemsPerSecond']= 1/(train['scannedLineItemsPerSecond'])
test['1/scannedLineItemsPerSecond']= 1/(test['scannedLineItemsPerSecond'])

##^2
train['lineItemVoidsPerPosition_squared'] = train['lineItemVoidsPerPosition']**2
test['lineItemVoidsPerPosition_squared'] = test['lineItemVoidsPerPosition']**2

##^3
train['valuePerSecond_cubed'] = train['valuePerSecond'] **3
test['valuePerSecond_cubed'] = test['valuePerSecond'] **3

##natural log
train['Naturallog_grandTotal'] = np.log(train['grandTotal'])
test['Naturallog_grandTotal'] = np.log(test['grandTotal'])

##log base 10
train['log10_totalScanTimeInSeconds'] = np.log10(train['totalScanTimeInSeconds'])
test['log10_totalScanTimeInSeconds'] = np.log10(test['totalScanTimeInSeconds'])

## >5
train['0_1_scansWithoutRegistration'] = np.where(train['scansWithoutRegistration'] < 5, 0, 1) 
test['0_1_scansWithoutRegistration'] = np.where(test['scansWithoutRegistration'] < 5, 0, 1) 

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
##engineering interactions based on feature importance from homework 4
train['interaction_1'] = train['newtrustLevel'] * train['trustLevel']
test['interaction_1'] = test['newtrustLevel'] * test['trustLevel']

train['interaction_2'] = train['newtrustLevel'] * train['1/scannedLineItemsPerSecond']
test['interaction_2'] = test['newtrustLevel'] * test['1/scannedLineItemsPerSecond']

train['interaction_3'] = train['trustLevel'] * train['1/scannedLineItemsPerSecond']
test['interaction_3'] = test['trustLevel'] * test['1/scannedLineItemsPerSecond']

In [6]:
## final interactions from homework 4 
##interaction 1 <= .862 for all
train['tree_interaction1'] = np.where(train['interaction_1'] <= 0.862, 1, 0)

## interaction 1 --> interaction 1 --> totalScanTimeInSeconds 
train['tree_interaction_2'] = np.where(((train['interaction_1'] >= .862) & 
                                     (train['interaction_1'] <= 3.204) & 
                                     (train['totalScanTimeInSeconds'] <= 1298)), 1, 0)

## interaction 1 (.862) --> 1/scannedLineItemsPerSecond (86.763) --> log10_totalScanTimeInSeconds (3.022)
train['tree_interaction_3'] = np.where(((train['interaction_1'] >= .862) & 
                                     (train['1/scannedLineItemsPerSecond'] <= 86.763) & 
                                     (train['log10_totalScanTimeInSeconds'] <= 3.022)), 1, 0)

## same things for test
test['tree_interaction1'] = np.where(test['interaction_1'] <= 0.862, 1, 0)

test['tree_interaction_2'] = np.where(((test['interaction_1'] >= .862) & 
                                     (test['interaction_1'] <= 3.204) & 
                                     (test['totalScanTimeInSeconds'] <= 1298)), 1, 0)

test['tree_interaction_3'] = np.where(((test['interaction_1'] >= .862) & 
                                     (test['1/scannedLineItemsPerSecond'] <= 86.763) & 
                                     (test['log10_totalScanTimeInSeconds'] <= 3.022)), 1, 0)

In [7]:
##creating lists
logit = list()
rf = list()
ada = list()

X = train.drop(columns = ['fraud'])
Y = train['fraud']

##loop for 100 iterations
for i in range (0,100):
    ##splitting the data 
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, stratify = Y)
    
## LOGISTIC REGRESSION ##
    Logit_rfe = RFE(estimator = LogisticRegression(), n_features_to_select = 5).fit(X_train, Y_train)
    logit.append(Logit_rfe.support_)
    
## RANDOM FOREST ##
    RF_rfe = RFE(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3), n_features_to_select = 5).fit(X_train, Y_train)
    rf.append(RF_rfe.support_)
    
## ADABOOST ##
    ADA_rfe = RFE(estimator = AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth=3), n_estimators=500, learning_rate=.01), n_features_to_select = 5).fit(X_train, Y_train)
    ada.append(ADA_rfe.support_)
    
    print(i)
    
##making dataframes
logit_results = pd.DataFrame(logit)
logit_results.columns = X_train.columns 

rf_results = pd.DataFrame(rf)
rf_results.columns = X_train.columns 

ada_results = pd.DataFrame(ada)
ada_results.columns = X_train.columns 

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

0


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

1


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

...

97


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

98


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules

99


In [8]:
logit_results

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemsVoids,scansWithoutRegistration,quanitityModification,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,newtrustLevel,...,valuePerSecond_cubed,Naturallog_grandTotal,log10_totalScanTimeInSeconds,0_1_scansWithoutRegistration,interaction_1,interaction_2,interaction_3,tree_interaction1,tree_interaction_2,tree_interaction_3
0,True,False,False,False,False,False,False,False,False,False,...,False,False,True,True,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,True,False
4,True,False,False,False,False,False,False,False,True,False,...,False,False,True,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,True,False,False,False,False,False,False,False,False,False,...,False,False,True,True,False,False,False,False,True,False
96,True,False,False,False,False,False,False,False,True,False,...,False,False,True,True,False,False,False,False,True,False
97,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,True,False
98,False,False,False,False,False,False,False,False,True,False,...,False,False,True,False,True,False,False,False,True,False


In [16]:
##putting totals for each
logit_feat = (100*logit_results.apply(np.sum, axis = 0)) / (logit_results.shape[0])
logit_feat 

trustLevel                           50.0
totalScanTimeInSeconds                0.0
grandTotal                            0.0
lineItemsVoids                        0.0
scansWithoutRegistration              0.0
quanitityModification                 0.0
scannedLineItemsPerSecond             1.0
valuePerSecond                        7.0
lineItemVoidsPerPosition             35.0
newtrustLevel                         0.0
1/scannedLineItemsPerSecond           0.0
lineItemVoidsPerPosition_squared     69.0
valuePerSecond_cubed                  0.0
Naturallog_grandTotal                 0.0
log10_totalScanTimeInSeconds        100.0
0_1_scansWithoutRegistration         82.0
interaction_1                        50.0
interaction_2                         0.0
interaction_3                         0.0
tree_interaction1                     0.0
tree_interaction_2                  100.0
tree_interaction_3                    6.0
dtype: float64

In [14]:
##putting totals for each
rf_feat = (100*rf_results.apply(np.sum, axis = 0)) / (rf_results.shape[0])
rf_feat 

trustLevel                          35.0
totalScanTimeInSeconds              61.0
grandTotal                           0.0
lineItemsVoids                       0.0
scansWithoutRegistration             0.0
quanitityModification                0.0
scannedLineItemsPerSecond           42.0
valuePerSecond                       0.0
lineItemVoidsPerPosition             7.0
newtrustLevel                       43.0
1/scannedLineItemsPerSecond         30.0
lineItemVoidsPerPosition_squared     7.0
valuePerSecond_cubed                 0.0
Naturallog_grandTotal                0.0
log10_totalScanTimeInSeconds        49.0
0_1_scansWithoutRegistration         0.0
interaction_1                       42.0
interaction_2                       73.0
interaction_3                       99.0
tree_interaction1                   12.0
tree_interaction_2                   0.0
tree_interaction_3                   0.0
dtype: float64

In [15]:
## putting totals for each 
ada_feat = (100*ada_results.apply(np.sum, axis = 0)) / (ada_results.shape[0])
ada_feat 

trustLevel                            0.0
totalScanTimeInSeconds               84.0
grandTotal                           12.0
lineItemsVoids                       87.0
scansWithoutRegistration             70.0
quanitityModification                 0.0
scannedLineItemsPerSecond             0.0
valuePerSecond                       15.0
lineItemVoidsPerPosition             13.0
newtrustLevel                         0.0
1/scannedLineItemsPerSecond           1.0
lineItemVoidsPerPosition_squared     11.0
valuePerSecond_cubed                  1.0
Naturallog_grandTotal                24.0
log10_totalScanTimeInSeconds         81.0
0_1_scansWithoutRegistration          0.0
interaction_1                         0.0
interaction_2                         1.0
interaction_3                       100.0
tree_interaction1                     0.0
tree_interaction_2                    0.0
tree_interaction_3                    0.0
dtype: float64

In [10]:
##all three tests combined
results = pd.concat([logit_feat, rf_feat, ada_feat], axis=1, join="inner")
results
## 0 is Logistic Regression
## 1 is Random Forest
## 2 Adaboost 

,0,1,2
trustLevel,50.0,35.0,0.0
totalScanTimeInSeconds,0.0,61.0,84.0
grandTotal,0.0,0.0,12.0
lineItemsVoids,0.0,0.0,87.0
scansWithoutRegistration,0.0,0.0,70.0
quanitityModification,0.0,0.0,0.0
scannedLineItemsPerSecond,1.0,42.0,0.0
valuePerSecond,7.0,0.0,15.0
lineItemVoidsPerPosition,35.0,7.0,13.0
newtrustLevel,0.0,43.0,0.0


In [13]:
##averages for the groups
results['Average'] = results.apply(np.mean, axis = 1)
results = results.sort_values(by = 'Average', ascending = False)
results

,0,1,2,Average
log10_totalScanTimeInSeconds,100.0,49.0,81.0,76.666667
interaction_3,0.0,99.0,100.0,66.333333
totalScanTimeInSeconds,0.0,61.0,84.0,48.333333
tree_interaction_2,100.0,0.0,0.0,33.333333
interaction_1,50.0,42.0,0.0,30.666667
lineItemVoidsPerPosition_squared,69.0,7.0,11.0,29.000000
lineItemsVoids,0.0,0.0,87.0,29.000000
trustLevel,50.0,35.0,0.0,28.333333
0_1_scansWithoutRegistration,82.0,0.0,0.0,27.333333
interaction_2,0.0,73.0,1.0,24.666667
